# PrimAITE Router Simulation Demo

This demo uses a modified version ARCD Use Case 2 Network (seen below) to demonstrate the capabilities of the Network simulator in PrimAITE.

## The Network
First let's create our network. The network comes 'pre-packaged' with PrimAITE in the `primaite.simulator.network.networks` module.

> ℹ️ You'll see a bunch of logs associated with parts of the Network that aren't an 'electronic' device on the Network and thus don't have a stream to log to. Soon these logs are going to be pushed to a Network Logger so we're not clogging up the PrimAITE application logs.

In [ ]:
from primaite.simulator.network.networks import network_simulator_demo_example

In [ ]:
network = network_simulator_demo_example()

Most of the Network components have a `.show()` function that prints a table of information about that object. We can view the Nodes and Links on the Network by calling `network.show()`.

In [ ]:
network.show()

## Nodes

Now let's inspect some of the nodes. We can directly access a node on the Network by calling .`get_node_by_hostname`. Like Network, a Node, along with some core services like ARP, have a `.show()` method.

### Router Nodes

First we'll inspect the Router node and some of it's core services.

Calling `router.show()` displays the Ethernet interfaces on the Router. If you need a table in markdown format, pass `markdown=True`.

In [ ]:
network.get_node_by_hostname("router_1").show()

Calling `router.arp.show()` displays the Router ARP Cache.

In [ ]:
network.get_node_by_hostname("router_1").arp.show()

Calling `router.acl.show()` displays the Access Control List.

In [ ]:
network.get_node_by_hostname("router_1").acl.show()

Calling `router.router_table.show()` displays the static routes the Router provides. 

In [ ]:
network.get_node_by_hostname("router_1").route_table.show()

Calling `router.sys_log.show()` displays the Router system log. By default, only the last 10 log entries are displayed, this can be changed by passing `last_n=<number of log entries>`.

NB: For `sys_log.show()` to work correctly log files need to be created with a sys_log level of INFO or below.

In [ ]:
network.get_node_by_hostname("router_1").sys_log.show()

### Switch Nodes

Next we'll inspect the Switch node and some of its core services.

Calling `switch.show()` displays the Switch ports on the Switch.

In [ ]:
network.get_node_by_hostname("switch_1").show()

Calling `switch.sys_log.show()` displays the Switch system log. By default; only the last 10 log entries are displayed, this can be changed by passing `last_n=<number of log entries>`.

In [ ]:
network.get_node_by_hostname("switch_1").sys_log.show()

### Computer/Server Nodes

Finally, we'll inspect a Computer or Server Node and some of its core services.

Calling `computer.show()` displays the NICs on the Computer/Server.

In [ ]:
network.get_node_by_hostname("security_suite").show()

Calling `computer.arp.show()` displays the Computer/Server ARP Cache.

In [ ]:
network.get_node_by_hostname("security_suite").arp.show()

Calling `computer.sys_log.show()` displays the Computer/Server system log. By default, only the last 10 log entries are displayed; this can be changed by passing `last_n=<number of log entries>`.

In [ ]:
network.get_node_by_hostname("security_suite").sys_log.show(last_n=25)

## Basic Network Comms Check

We can perform a good old ping to check that Nodes are able to communicate with each other.

In [ ]:
network.show(nodes=False, links=False)

We'll first ping client_1's default gateway.

In [ ]:
network.get_node_by_hostname("client_1").ping("192.168.10.1")

In [ ]:
network.get_node_by_hostname("client_1").sys_log.show(last_n=15)

Next, we'll ping the interface of the 192.168.1.0/24 Network on the Router (port 1).

In [ ]:
network.get_node_by_hostname("client_1").ping("192.168.1.1")

And finally, we'll ping the web server.

In [ ]:
network.get_node_by_hostname("client_1").ping("192.168.1.12")

To confirm that the ping was received and processed by the web_server, we can view the sys log

In [ ]:
network.get_node_by_hostname("web_server").sys_log.show()

## Advanced Network Usage

We can now use the Network to perform some more advanced things.

Let's attempt to prevent client_2 from being able to ping the web server. First, we'll confirm that it can ping the server...

In [ ]:
network.get_node_by_hostname("client_2").ping("192.168.1.12")

If we look at the client_2 sys log we can see that the four ICMP echo requests were sent and four ICMP each replies were received.

In [ ]:
network.get_node_by_hostname("client_2").sys_log.show()

Now we'll add an ACL to block ICMP from 192.168.10.22.

In [ ]:
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port
from primaite.simulator.network.hardware.nodes.network.router import  ACLAction
network.get_node_by_hostname("router_1").acl.add_rule(
    action=ACLAction.DENY,
    protocol=IPProtocol.ICMP,
    src_ip_address="192.168.10.22",
    position=1
)

In [ ]:
network.get_node_by_hostname("router_1").acl.show()

Now we attempt (and fail) to ping the web server.

In [ ]:
network.get_node_by_hostname("client_2").ping("192.168.1.12")

We can check that the ping was actually sent by client_2 by viewing the sys log.

In [ ]:
network.get_node_by_hostname("client_2").sys_log.show()

We can check the router sys log to see why the traffic was blocked.

In [ ]:
network.get_node_by_hostname("router_1").sys_log.show()

Now a final check to ensure that client_1 can still ping the web_server.

In [ ]:
network.get_node_by_hostname("client_1").ping("192.168.1.12")

In [ ]:
network.get_node_by_hostname("client_1").sys_log.show()